In [1]:
# 인퍼런스 설정
from ratsnlp.nlpbook.ner import NERDeployArguments
args = NERDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="nlpbook/ner",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: nlpbook/ner\epoch=1-val_loss=0.20.ckpt
downstream_model_labelmap_fpath: nlpbook/ner\label_map.txt


In [2]:
# 토크나이저 로드
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [3]:
# 체크포인트 로드
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)

In [4]:
# BERT 설정 로드
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt["state_dict"]["model.classifier.bias"].shape.numel(),
)

In [5]:
# BERT 모델 초기화
from transformers import BertForTokenClassification
model = BertForTokenClassification(pretrained_model_config)

In [6]:
# 체크포인트 주입
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt["state_dict"].items()})

<All keys matched successfully>

In [7]:
# 평가 모드로 전환
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [8]:
# 레이블 맵 작성
labels = [label.strip() for label in open(args.downstream_model_labelmap_fpath, "r").readlines()]
id_to_label = {}
for idx, label in enumerate(labels):
  if "PER" in label:
    label = "인명"
  elif "LOC" in label:
    label = "지명"
  elif "ORG" in label:
    label = "기관명"
  elif "DAT" in label:
    label = "날짜"
  elif "TIM" in label:
    label = "시간"
  elif "DUR" in label:
    label = "기간"
  elif "MNY" in label:
    label = "통화"
  elif "PNT" in label:
    label = "비율"
  elif "NOH" in label:
    label = "기타 수량표현"
  elif "POH" in label:
    label = "기타"
  else:
    label = label
  id_to_label[idx] = label

In [9]:
id_to_label

{0: '[CLS]',
 1: '[SEP]',
 2: '[PAD]',
 3: '[MASK]',
 4: 'O',
 5: '인명',
 6: '기타 수량표현',
 7: '기타',
 8: '기관명',
 9: '날짜',
 10: '지명',
 11: '통화',
 12: '비율',
 13: '시간',
 14: '기간',
 15: '인명',
 16: '기타 수량표현',
 17: '기타',
 18: '기관명',
 19: '날짜',
 20: '지명',
 21: '통화',
 22: '비율',
 23: '시간',
 24: '기간'}

In [10]:
# 인퍼런스
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )

    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k ,v in inputs.items()})
        probs = outputs.logits[0].softmax(dim=1)
        # 각 토큰이 속하는 개체명 확률 분포 가운데 가장 높은 확률값과 그에 속하는 인덱스 구하기
        top_probs, preds = torch.topk(probs, dim=1, k=1)
        # 토큰 인덱스 시퀀소를 토큰 시퀀스로 변환
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        # 개체명 인덱스 시퀀스를 개체명 시퀀스로 변환
        predicted_tags = [id_to_label[pred.item()] for pred in preds]
        result = []
        for token, predicted_tag, top_prob in zip(tokens, predicted_tags, top_probs):
            if token not in [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token]:
                token_result ={
                    "token": token,
                    "predicted_tag": predicted_tag,
                    "top_prob": str(round(top_prob[0].item(), 4)),
                }
                result.append(token_result)
    return {
        "sentence": sentence,
        "result": result,
    }

In [11]:
# 웹 서비스
from ratsnlp.nlpbook.ner import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app 'ratsnlp.nlpbook.ner.deploy'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


 * Running on http://5887-211-204-110-53.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/Feb/2023 21:58:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2023 21:58:53] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2023 21:59:08] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2023 21:59:17] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2023 21:59:23] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2023 21:59:27] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2023 21:59:34] "POST /api HTTP/1.1" 200 -
